In [1]:
import numpy as np


In [2]:
#je construis mon data set d'entrainement avec la methode xor
X = np.random.randint(2, size = (2, 4)) #mon dataset est constitué de 2 exemples(0 et 1), sa taille est 2 lignes et 400 colonnes
Y = np.logical_xor(X[0, :], X[1, :]) # Y est mon label

In [3]:
Y = Y.reshape(1, 4)
print("X.shape = ", X.shape)
print("Y.shape = ", Y.shape)

X.shape =  (2, 4)
Y.shape =  (1, 4)


Architecture de mon reseau de neurone

In [4]:
architecture = [2, 3, 3, 1]
def initialisation_param(architecture):
    np.random.seed(1)
    couches = len(architecture)# nombre de layers y compris le input layer. Or concretement, ce dernier ne fait pas parti de notre réseau de neurone
    parametres = {}
    
    for l in range(1, couches):
        parametres["W" + str(l)] = np.random.randn(architecture[l],architecture[l-1])*0.01
        parametres["b" + str(l)] = np.zeros((architecture[l], 1))
    
    return parametres

In [5]:
parametres = initialisation_param(architecture)
parametres

{'W1': array([[ 0.01624345, -0.00611756],
        [-0.00528172, -0.01072969],
        [ 0.00865408, -0.02301539]]), 'b1': array([[0.],
        [0.],
        [0.]]), 'W2': array([[ 0.01744812, -0.00761207,  0.00319039],
        [-0.0024937 ,  0.01462108, -0.02060141],
        [-0.00322417, -0.00384054,  0.01133769]]), 'b2': array([[0.],
        [0.],
        [0.]]), 'W3': array([[-0.01099891, -0.00172428, -0.00877858]]), 'b3': array([[0.]])}

In [6]:

parametres["W" + str(1)]

array([[ 0.01624345, -0.00611756],
       [-0.00528172, -0.01072969],
       [ 0.00865408, -0.02301539]])

In [7]:
def sigmoid(z):
    
    S = 1/(1+ np.exp(-z))
    return S

In [8]:
len(parametres)
len(architecture)-1

3

In [9]:
def forward(X, parametres):
    caches = {} #creation de dictionnaire vide pour stocker les resultats 
    caches["A0"] = X #A0 = X
    
    param= len(parametres)//2  
    
    for i in range(1,param+1):
        
        A = caches["A" + str(i - 1)] 
    
        W = parametres["W" + str(i)]
        
        b = parametres["b" + str(i)]

        Z = np.dot(W,A) + b

        A = sigmoid(Z) #A1

        caches["A" + str(i)] = A #on enregistre A1 dans le dict

#         Z1 = np.dot(W1, X)+b1
#         A1 = sigmoid(Z1)
#         Z2 = np.dot(W2, A1)+b2
#         A2 = sigmoid(Z2)
#         Z3 = np.dot(W3, A2)+b3
#         A3 = sigmoid(Z3)

    return A, caches

In [10]:
A, caches = forward(X, parametres)
A
caches

{'A0': array([[1, 0, 0, 0],
        [1, 1, 1, 0]]),
 'A1': array([[0.50253145, 0.49847061, 0.49847061, 0.5       ],
        [0.49599723, 0.4973176 , 0.4973176 , 0.5       ],
        [0.49640973, 0.49424641, 0.49424641, 0.5       ]]),
 'A2': array([[0.5016441 , 0.50162214, 0.50162214, 0.5016283 ],
        [0.49894303, 0.49896153, 0.49896153, 0.49894075],
        [0.50052575, 0.50052162, 0.50052162, 0.50053412]]),
 'A3': array([[0.49730708, 0.49730715, 0.49730715, 0.49730711]])}

In [11]:
def cost(A, Y):
    m = Y.shape[1]
    L = np.multiply(np.log(A), Y)+(1-Y)* np.log(1-A)
    cost = - np.sum(L)/4
    
    cost = np.squeeze(cost)
    return cost

In [12]:
cost(A , Y)

0.6931616355342991

In [13]:
caches['A0']

array([[1, 0, 0, 0],
       [1, 1, 1, 0]])

In [14]:
def backward(parametres, caches, Y ):
    """
    Backward propagation sans boucle for.
    :m = nombres d'exemples
    
    """
    m = X.shape[1]
    #J'extrais les poids du dictionnaire parametres, en faisant appel aux clés associées
    W1= parametres['W1']
    W2= parametres['W2']
    W3 = parametres['W3']
    
    #j'extrais A: activations de chaque neurone
    A1 = caches['A1']
    A2 = caches['A2']
    A3 = caches['A3']
    A0 = caches['A0']
    
    #calculs de dérivées pour la backward
    dZ3 = A3 - Y
    
    dW3 = np.dot(dZ3, A2.T)
    db3 = np.sum(dZ3, axis = 1, keepdims = True)/m 
    
    dA2 = np.dot(W3.T,dZ3)     # derivée de l'activation A2
    dZ2 = dA2 * A3 * (1 - A3)  # derivée de la sigmoid Z2
    dW2 = np.dot(dZ2, A2.T)    # dérivée du poids w3
    db2 = np.sum(dZ2, axis =1, keepdims = True)/m
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * A2 * (1 - A2)
    dW1 = np.dot(dZ1, A0.T)
    db1 = np.sum(dZ1, axis =1, keepdims = True)/m
    
    grads = {"dW1" : dW1,
            "db1" : db1,
            "dW2" : dW2,
            "db2" : db2,
            "dW3": dW3,
            "db3": db3}
    
    return grads

In [15]:
def backward_pro(parametres, caches, Y):
    """
    Utilisation de boucle for pour automatiser le code.
    """
    
    grads = {}
    couches = len(parametres//2)
    grads["dZ"] = caches["A" + str(couches)]-Y
    
    for i in reversed(range(couches+1)):
        W = parametres["W" + str(i)]
        A = caches["A" + str(i-1)]
        dZ = grads ["dZ" + str(i)]
        
        #calculs dérivées
        dA = np.dot(W.T,dZ)
        dZ = dA * A * (1 - A)
        dW = np.dot(dZ, A.T)
        db = np.sum(dZ, axis =1, keepdims = True)/m
        
        grads["dW" + str(i)] = dW
        grads["dZ" + str(i)] = dZ
        grads["db" + str(i)] = db
       
    
        return grads
        

In [16]:
grads = backward(parametres, caches, Y)
grads

{'dW1': array([[-4.95132810e-06,  5.05857542e-06],
        [ 2.86651113e-06, -2.92859990e-06],
        [-3.08001285e-06,  3.14672522e-06]]), 'db1': array([[ 2.68115103e-08],
        [-1.55221686e-08],
        [ 1.66780793e-08]]), 'dW2': array([[1.48185479e-05, 1.48318942e-05, 1.48016518e-05],
        [2.32308022e-06, 2.32517250e-06, 2.32043146e-06],
        [1.18271573e-05, 1.18378094e-05, 1.18136720e-05]]), 'db2': array([[7.40446874e-06],
        [1.16078681e-06],
        [5.90974345e-06]]), 'dW3': array([[-0.00538925, -0.00539411, -0.00538311]]), 'db3': array([[-0.00269288]])}

In [17]:
def update_parametres(parametres, grads, l_r=1.2):
    
    parametres["W1"] = parametres["W1"]-l_r * grads["dW1"]
    parametres["W2"] = parametres["W2"]-l_r * grads["dW2"]
    parametres["W3"] = parametres["W3"]-l_r * grads["dW3"]
    parametres["b1"] = parametres["b1"]-l_r * grads["db1"]
    parametres["b2"] = parametres["b2"]-l_r * grads["db2"]
    parametres["b3"] = parametres["b3"]-l_r * grads["db3"]
    
    return parametres

In [18]:
update_parametres(parametres, grads, l_r=1.2)

{'W1': array([[ 0.0162494 , -0.00612363],
        [-0.00528516, -0.01072617],
        [ 0.00865777, -0.02301916]]), 'b1': array([[-3.21738123e-08],
        [ 1.86266024e-08],
        [-2.00136951e-08]]), 'W2': array([[ 0.01743034, -0.00762987,  0.00317263],
        [-0.00249649,  0.01461829, -0.02060419],
        [-0.00323836, -0.00385475,  0.01132352]]), 'b2': array([[-8.88536249e-06],
        [-1.39294417e-06],
        [-7.09169214e-06]]), 'W3': array([[-0.00453181,  0.00474864, -0.00231886]]), 'b3': array([[0.00323145]])}

In [19]:
def update_p_with_for(parametres, grads, learning_rate = 1.2):

    param = len(parametres)//2
    
    for p in range(param):
        parametres["W" + str(p+1)] = parametres["W" + str(p+1)] -learning_rate * grads["dW" + str(p+1)]
        parametres["b" + str(p+1)] = parametres["b" + str(p+1)] - learning_rate * grads["db"+ str(p+1)]
        
        return parametres

In [20]:
update_p_with_for(parametres, grads, learning_rate = 1.2)

{'W1': array([[ 0.01625534, -0.0061297 ],
        [-0.0052886 , -0.01072266],
        [ 0.00866147, -0.02302294]]), 'b1': array([[-6.43476247e-08],
        [ 3.72532047e-08],
        [-4.00273903e-08]]), 'W2': array([[ 0.01743034, -0.00762987,  0.00317263],
        [-0.00249649,  0.01461829, -0.02060419],
        [-0.00323836, -0.00385475,  0.01132352]]), 'b2': array([[-8.88536249e-06],
        [-1.39294417e-06],
        [-7.09169214e-06]]), 'W3': array([[-0.00453181,  0.00474864, -0.00231886]]), 'b3': array([[0.00323145]])}

In [21]:
# def update_param_with_for(parametres, grads, learning_rate=1.2):
    
#     parametres = {}
#     print("parametres", parametres )
    
#     p =len(parametres) 
#     print("p", p)
#     parametres["W1"] = grads["dW" + str(1)]
#     print(parametres)
    
#     for i in reversed(range(1, p +1)):
#         print("EST CE QUE J'EXISTE???? :()")
#         dW = grads["dW" + str(i)]
#         db = grads["db" + str(i)]
#         W = parametres["W" + str(i)]- learning_rate * dW
        
#         b = parametres["b" + str(i)]- learning_rate * db
        
#     return parametres

In [22]:
# parametres = update_param_with_for(parametres, grads, learning_rate=1.1)
# parametres


In [23]:
def neural_network_model(X, Y, nombre_iteration = 1000, l print_cost = False):
    
    n_x = (X, Y)[0]
    n_y = (X, Y)[1]*1
   
    parametres = initialisation_param(architecture)
    W1 = parametres["W1"]
    W2 = parametres["W2"]
    w3 = parametres["W3"]
    b1 = parametres["b1"]
    b2 = parametres["b2"]
    b3 = parametres["b3"]
    
    for i in range(0, nombre_iteration):
        A3, caches = forward(X, parametres)
        cost = cost(A3, Y,parametres)
        grads = backward_pro(parametres, caches, X, Y)
        parametres = update_parametres(parametres, grads, learning_rate=1.2)
        
        if print_cost and i% 1000 == 0:
            print("cost after iteration %i : %f" %(i,cost))
    
    return parametres
    
    

SyntaxError: invalid syntax (<ipython-input-23-449a48dc482c>, line 1)

In [ ]:
neural_network_model(X, Y, nombre_iteration = 1000, print_cost = False)

In [ ]:
p = len(parametres)
p

In [ ]:
for i in reversed(range(1, p +1)):
    print(i)

In [ ]:
(X, Y)[1]*1